In [ ]:
import openai
import os

# from dotenv import load_dotenv, find_dotenv

# _ = load_dotenv(find_dotenv())

# openai.api_key = os.getenv('OPENAI_API_KEY')
openai.api_key = os.environ["OPENAI_API_KEY"]

new_line = '\n'

In [ ]:
import pandas as pd
df = pd.read_csv ('sign_events_data_statements.csv')
# simple EDA
print(df)
print(df.columns)
print(f'shape: {df.shape}')  # should be 1640 x 6


def filter_by_class(df, category):
   return df[df['outcome_class']== category]

truth_df = filter_by_class(df, 't')
# print(truth_df)
print(f'truth df shape: {truth_df.shape}')  # should be 1640 x 6

# replace with a more expressive word, truthful
truth_df['outcome_class'] = df['outcome_class'].replace('t','truthful')
print(truth_df)

deceit_df = filter_by_class(df, 'd')
# print(deceit_df)
print(f'deceit df shape: {deceit_df.shape}')  # should be 1640 x 6

# replace with a more expressive word, deceitful
deceit_df['outcome_class'] = df['outcome_class'].replace('d','deceitful')
print(deceit_df)

# pick random non-repeating rows
def randon_non_repeating(df, quantity):
    import random
    rand_df = pd.DataFrame()
    random_list = random.sample(range(df.shape[0]), quantity)
    print("non-repeating random numbers are:")
    return df.iloc[random_list]

nb_samples_of_each = 5
random_truth_df = randon_non_repeating(truth_df, nb_samples_of_each)
print(f'random truth list:\n, {random_truth_df}')

random_deceit_df = randon_non_repeating(deceit_df, nb_samples_of_each)
print(f'random deceit list:\n, {random_deceit_df}')

random_truth_deceit_df = pd.concat([random_truth_df, random_deceit_df])



In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [18]:
intro = f"""
You are a detective presented with the desciption of an \
activity and a supporting statement to reassure you that \
the description is truthful. You have to decide whether the \
description is truthful or deceitful.
"""

def construct_scenario(row):
    event_header = 'Event:'
    activity_description_header = 'Description of activity by person:'

    activity_reassurance_header = 'Information to reassure you that the person is telling the truth about their activity:'

    event = event_header + new_line + row['signevent'] + new_line
    q1 = activity_description_header + new_line + row['q1'] + new_line
    q2 = activity_reassurance_header + new_line + row['q2'] + new_line
    return event + q1 + q2

def construct_outcome(row):
    outcome = "Your Judgement:"
    return outcome + new_line + row['outcome_class'] + new_line * 2
    

def construct_few_shot_prompt(df, infer_row):
    messages = []
    messages.append(intro)

    for _, row in df.iterrows():
        messages.append(construct_scenario(row))
        messages.append(construct_outcome(row))
    
    messages.append(construct_scenario(infer_row))
    messages.append(construct_outcome(infer_row)) # has to have a black outcome to be filled by the llm
    return messages

infer_row = df.iloc[1171]  # pick a random row
print(f'Inferring the `class_outcome` for:\n{infer_row}')
ground_truth = 'truthful' if infer_row['outcome_class'] == 't' else 'deceitful'
# mask the `outcome_class` field since you want to predict it
infer_row['outcome_class'] = ''

prompt = construct_few_shot_prompt(random_truth_deceit_df, infer_row=infer_row)
prompt = ''.join(prompt)



Inferring the `class_outcome` for:
signevent        I am going to meet with my father in law who i...
q1                I have just had a great phone call from my hu...
q2               I can give you details of the phone call\nscre...
unid                                                      rj485773
id                                                            1172
outcome_class                                                    d
Name: 1171, dtype: object


/tmp/ipykernel_6567/2763430961.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  infer_row['outcome_class'] = ''


In [20]:
print(f'{prompt}')
MODEL = "gpt-3.5-turbo"
response = get_completion(
    prompt=prompt,
    model=MODEL,
)    
        
print(f'ENTIRE RESPONSE:\n {response}')




You are a detective presented with the desciption of an activity and a supporting statement to reassure you that the description is truthful. You have to decide whether the description is truthful or deceitful.
Event:
On Thursday at 9:20 am, I will go to the gym and undertake a class called outdoor fit. It is 50 minutes long.
Description of activity by person:
When  I get to my class, we normally warm-up for about 10 minutes, we then do various running exercises and use equipment such as skipping ropes and bands to improve our workout.
Information to reassure you that the person is telling the truth about their activity:
Withdean Gym in Brighton holds an outdoor fit class at the exact time I mentioned, it lasts for the correct period of time and involves all the activity that I have set out.
Your Judgement:
truthful

Event:
I am going on holiday today with my family. We will be going for 9 days. 
Description of activity by person:
I am going on a last minute holiday to Fuertaventura. 